In [ ]:
#import 
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import timm

: 

In [6]:
import sys
print(sys.executable)


c:\Users\rtvan\anaconda3\python.exe


In [1]:
# Root folder waar de afbeeldingen staan
root = r"C:\Users\rtvan\Documents\GitHub\AI-Waste_Vision_POC"
csv_path = root + r"\labels.csv"

df = pd.read_csv(csv_path)

# Relatieve → absolute paden
df["abs_path"] = df["image_path"].apply(lambda p: os.path.join(root, p))

# Label mapping: -1, 0, 1 → 0, 1, 2
df["label_idx"] = df["label"].map({-1: 0, 0: 1, 1: 2})

df.head()


NameError: name 'pd' is not defined

In [ ]:
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label_idx"],
    random_state=42
)

len(train_df), len(val_df)


In [ ]:
class ContainerDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row["abs_path"]
        label = int(row["label_idx"])

        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, label


In [ ]:
# ImageNet normalisatie (vereist door EfficientNet)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = ContainerDataset(train_df, train_transform)
val_dataset = ContainerDataset(val_df, val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = timm.create_model(
    "efficientnet_b0",
    pretrained=True,
    num_classes=3  # 3 klassen: -1, 0, 1 → gemapt naar 0, 1, 2
)

model = model.to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    correct = 0

    for imgs, labels in tqdm(loader):
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(dim=1) == labels).sum().item()

    accuracy = correct / len(loader.dataset)
    return total_loss / len(loader), accuracy


def validate(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    correct = 0

    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(dim=1) == labels).sum().item()

    accuracy = correct / len(loader.dataset)
    return total_loss / len(loader), accuracy


In [ ]:
best_val_acc = 0

for epoch in range(10):  # verhoog naar 20–30 als nodig
    print(f"\nEpoch {epoch+1}")

    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)

    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "efficientnet_best.pth")
        print("Saved new best model!")


In [ ]:
model.load_state_dict(torch.load("efficientnet_best.pth"))
model.eval()

# Mapping terug naar originele labels
reverse_map = {0: -1, 1: 0, 2: 1}

def predict(path):
    img = Image.open(path).convert("RGB")
    img = val_transform(img).unsqueeze(0).to(device)
    output = model(img)
    pred_idx = output.argmax(dim=1).item()
    return reverse_map[pred_idx]

# voorbeeld:
# predict(r"C:\Users\rtvan\Documents\GitHub\AI-Waste_Vision_POC\ProjectData\...\image.jpg")
